In [1]:
import psycopg2

def get_svv_connection():
    con = psycopg2.connect(dbname = 'svv', host ='svv-rs-prod-bi.cjddijbnvfpr.eu-west-1.redshift.amazonaws.com', \
                           port = 5439, user = 'ryanw', password = 'hT6Y3TeZZUFdvLj')
    
    return con

In [2]:
import pandas as pd

In [3]:
query_data = "Select * from sandbox.RW_hub_plus_segmentation_lifelines"

df_survival = pd.DataFrame()

try: 
    con = get_svv_connection()

    cur = con.cursor()
    cur.execute(query_data)

    colnames = [desc[0] for desc in cur.description]
    df_survival = pd.DataFrame(cur.fetchall(), columns=colnames)
    
except Exception as inst:
    print(inst)
finally:
    cur.close()
    con.close()

In [4]:
df_survival.head()

,viewer_id,tenure,end_date,churned
0,03274707-7f95-48b9-8a20-6c1575818ef3,1.0,2017-07-30 22:21:00,1
1,03a7e588-28d3-4893-9f8f-3a320215e071,12.0,2018-03-02 21:12:12,1
2,08096e31-1c25-4bab-82ff-4bdf104609f3,1.0,2017-08-09 23:20:37,1
3,08b743bc-bb01-457c-a14a-224673540ce4,1.0,2017-08-15 03:34:12,1
4,0b75c082-67cd-4707-9eb9-1561657ddd36,2.0,2018-03-19 01:42:31,1


In [5]:
df_survival = df_survival.loc[:,['viewer_id']]

In [6]:
df_clusters = pd.read_csv('cluster_simple_261018.csv')

In [8]:
df_clusters = df_clusters.loc[:,['viewer_id','clusters']]

In [9]:
# Join viewer data and demographics
from functools import reduce

df_survival2 = reduce(lambda left,right: pd.merge(left,right,on='viewer_id'), 
                  [df_survival, df_clusters])

df_survival2.head()

,viewer_id,clusters
0,03a7e588-28d3-4893-9f8f-3a320215e071,1
1,08096e31-1c25-4bab-82ff-4bdf104609f3,2
2,08b743bc-bb01-457c-a14a-224673540ce4,2
3,0b75c082-67cd-4707-9eb9-1561657ddd36,4
4,0d8452d6-98aa-44de-906b-6929658b0e5d,2


In [11]:
df_current = pd.read_csv('Current Active Hub+ IDs.csv')

In [12]:
df_cluster2 = reduce(lambda left,right: pd.merge(left,right,on='viewer_id'), 
                  [df_survival2, df_current])

df_cluster2.head()

,viewer_id,clusters
0,284aec81-e1df-4dc2-ba06-86e8623105cc,2
1,5b644f6e-4b37-40e6-a676-94af1db31acd,4
2,f79c4dd2-81e2-41cf-b1e6-f3de5638d243,0
3,c6c9a8cf-b637-4a38-af8c-181b86109487,0
4,0b78719d-edaa-4f78-be4d-29a357e85476,4


In [15]:
df_cluster2.groupby('clusters').size()

clusters
0    15355
1    14552
2    57390
3    19470
4    15914
dtype: int64